In [ ]:
import ee
import geemap

ee.Initialize()

## The map function

In [ ]:
mylist = ee.List.sequence(1, 10)
mylist

In [ ]:
def square(number):
    return ee.Number(number).pow(2)


squares = mylist.map(square)
squares

In [ ]:
def square(number):
    return ee.Number(number).pow(2).subtract(1)


squares = mylist.map(square)
squares

In [ ]:
## Lambda function function (only apply for one line function)

squares = mylist.map(lambda number: ee.Number(number).pow(2).subtract(1))
squares

### same in python without ee

In [ ]:
for i in range(1, 10):
    print(i)

In [ ]:
squares = []
for i in range(1, 11):
    squares.append(i**2)
squares

In [ ]:
[i**2 for i in range(1, 11)]

In [ ]:
Map = geemap.Map()

collection = (
    ee.ImageCollection("MODIS/MCD43A4_006_NDVI")
    .filter(ee.Filter.date("2018-06-01", "2018-07-01"))
    .select("NDVI")
)
vis_params = {
    "min": 0.0,
    "max": 1.0,
    "palette": "ndvi",
}

Map.add_time_slider(collection, vis_params, time_interval=2)
Map

In [ ]:
Map.user_rois

In [ ]:
images = collection.map(lambda img: img.clipToCollection(Map.user_rois))

In [ ]:
Map = geemap.Map()

vis_params = {
    "min": 0.0,
    "max": 1.0,
    "palette": "ndvi",
}

Map.add_time_slider(images, vis_params, time_interval=2)
Map

In [ ]:
Map = geemap.Map()
fc = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017").filter(
    ee.Filter.eq("country_na", "Netherlands")
)

Map.addLayer(fc, {"color": "ff000000"}, "Netherlands")
Map.centerObject(fc)
Map

In [ ]:
years = ee.List.sequence(2013, 2020)

In [ ]:
def year_image(year):
    start_date = ee.Date.fromYMD(year, 1, 1)
    end_date = start_date.advance(1, "year")
    collection = (
        ee.ImageCollection("LANDSAT/LC08/C02/T1")
        .filterBounds(fc)
        .filterDate(start_date, end_date)
    )
    image = ee.Algorithms.Landsat.simpleComposite(collection).clipToCollection(fc)

    return image

In [ ]:
images = years.map(year_image)

In [ ]:
images

In [ ]:
vis_params = {
    "bands": ["B4", "B3", "B2"],
    "max": 128,
}

for index in range(0, 8):
    image = ee.Image(images.get(index))
    layername = "Year" + str(index + 2013)
    Map.addLayer(image, vis_params, layername)

Map